In [109]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler
from sklearn.cluster import AgglomerativeClustering
import gower
from scipy.cluster.hierarchy import dendrogram

__DATENVORBEREITUNG__

In [110]:
# load data
artikel = pd.read_excel(r"C:\Users\IM-KI\Documents\PrEvelop_Daten_mkPlast\PrEvelOp_mkPlast_01_Esb.xlsx")
arbeitsplanzeilen = pd.read_excel(r"C:\Users\IM-KI\Documents\PrEvelop_Daten_mkPlast\PrEvelOp_mkPlast_01_Esb_Arbeitszeilen.xlsx")

In [111]:
# set identifier
artikel.rename(columns={'Nr.': 'index'}, inplace=True)
artikel.set_index('index', inplace=True)
display(artikel.head(5))
arbeitsplanzeilen.rename(columns={'Arbeitsplannr.': 'index'}, inplace=True)
arbeitsplanzeilen.set_index('index', inplace=True)
display(arbeitsplanzeilen.head(5))

,Beschreibung,Beschreibung 2,Mengeneinheit,Rohstoff,Artikelgruppe,Produktgruppe,Foliencode,Farbcode,Arbeitsplannr.,Breite(mm),...,Rollendurchmesser(mm),Rollenlänge(m),Stk pro Rolle,Randbeschnitt(mm),Bündelung(stk),Gewicht / 1000m,Echt Gewicht / 1000m,Gewicht / 1000skt,Echt Gewicht / 1000skt,MFI
index,,,,,,,,,,,,,,,,,,,,,
100012,MDPE Seitenfaltenbeutel,-Ascalon Umbeutel-,STK,MDPE,KONFEKTION,SFB,MT-LOS-,TRANSPARENT,100012,180,...,0,0.0,0,0,0,22.5120,22.463061,10.580640,10.557639,0.0
100013,HDPE Seitenfaltenbeutel,NaN,STK,HDPE,KONFEKTION,SFB,BL-,BLAU DUNKEL-2,100013,580,...,0,0.0,0,0,50,19.4616,0.000000,12.844656,0.000000,0.0
100014,LDPE Flachfolienzuschnitte,NaN,STK,LDPE,KONFEKTION,ZUS,ST-,TRANSPARENT,100014,600,...,0,0.0,0,0,500,11.0880,0.000000,7.761600,0.000000,0.0
100015,HDPE Seitenfaltenbeutel,NaN,STK,HDPE,KONFEKTION,SFB,LME-ST-,BLAU HELL-2,100015,400,...,0,0.0,0,0,500,13.3560,13.340224,4.674600,4.669078,0.0
100017,MDPE Flachbeutel,NaN,STK,MDPE,KONFEKTION,FB,LME-LOS-,ROT 5500-3,100017,395,...,0,0.0,0,0,0,33.3459,32.936424,24.675966,24.372954,0.0


,Versionscode,Arbeitsgangnr.,Nächste Arbeitsgangnr.,Vorherige Arbeitsgangnr.,Art,Nr.,Arbeitsplatzgruppennr.,Abteilungscode,Beschreibung,Rüstzeit,...,Reihenfolge (Rückwärts),Kum. feste Ausschussmenge,Kum. Ausschussfaktor %,Takt,kalk. Durchsatz,kalk. Bearbeitungszeit,Durchsatz,Nutzen (Bahnigkeit),Nutzen (Lagigkeit),Abzugsgeschwindigkeit
index,,,,,,,,,,,,,,,,,,,,,
100012,NaN,100,NaN,NaN,Arbeitsplatzgruppe,21.0,21.0,KONFEKTION,BSZ II,0.5,...,1,0,0.0,0.0,0.0,0.0,25000.0,3,1,0.0
100013,NaN,100,NaN,NaN,Arbeitsplatzgruppe,21.0,21.0,KONFEKTION,BSZ II,0.5,...,1,0,0.0,0.0,0.0,0.0,0.0,2,1,0.0
100014,NaN,100,NaN,NaN,Arbeitsplatzgruppe,20.0,20.0,KONFEKTION,BSZ I,0.5,...,1,0,0.0,0.0,0.0,0.0,0.0,2,4,0.0
100015,NaN,100,NaN,NaN,Arbeitsplatzgruppe,21.0,21.0,KONFEKTION,BSZ II,0.5,...,1,0,0.0,140.0,0.0,0.0,16800.0,2,1,0.0
100016,NaN,100,NaN,NaN,Arbeitsplatzgruppe,3.0,3.0,EXTRUSION,EX3,0.5,...,1,0,0.0,0.0,0.0,0.0,0.0,1,1,0.0


In [112]:
# check if identifier is unique
print(artikel.index.is_unique)
print(arbeitsplanzeilen.index.is_unique)
# count values in Arbeitsplannr.
print(arbeitsplanzeilen.index.value_counts())

True
False
index
104148    2
100012    1
301214    1
301204    1
301205    1
         ..
105753    1
105754    1
105755    1
105756    1
77        1
Name: count, Length: 13926, dtype: int64


In [113]:
# drop row in arbeitsplatzeilen with duplicate index
arbeitsplanzeilen = arbeitsplanzeilen[~arbeitsplanzeilen.index.duplicated(keep='first')]


In [114]:
# show indices in arbeitsplanzeilen that do not appear in artikel
arbeitsplanzeilen[~arbeitsplanzeilen.index.isin(artikel.index)==False]

,Versionscode,Arbeitsgangnr.,Nächste Arbeitsgangnr.,Vorherige Arbeitsgangnr.,Art,Nr.,Arbeitsplatzgruppennr.,Abteilungscode,Beschreibung,Rüstzeit,...,Reihenfolge (Rückwärts),Kum. feste Ausschussmenge,Kum. Ausschussfaktor %,Takt,kalk. Durchsatz,kalk. Bearbeitungszeit,Durchsatz,Nutzen (Bahnigkeit),Nutzen (Lagigkeit),Abzugsgeschwindigkeit
index,,,,,,,,,,,,,,,,,,,,,


In [115]:
artikel.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16262 entries, 100012 to WA00008
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Beschreibung            16234 non-null  object 
 1   Beschreibung 2          2736 non-null   object 
 2   Mengeneinheit           16262 non-null  object 
 3   Rohstoff                15317 non-null  object 
 4   Artikelgruppe           16251 non-null  object 
 5   Produktgruppe           15544 non-null  object 
 6   Foliencode              14459 non-null  object 
 7   Farbcode                15194 non-null  object 
 8   Arbeitsplannr.          15165 non-null  object 
 9   Breite(mm)              16262 non-null  int64  
 10  Seitenfalte(mm)         16262 non-null  int64  
 11  Länge(mm)               16262 non-null  int64  
 12  Stärke(mm)              16262 non-null  float64
 13  Echte Stärke(mm)        16262 non-null  float64
 14  Dichte(g/cm³)           16262 non-nu